In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
# Dict for leagues
league_dict = {
    # Name of leagues
    "league": ["PremierLeague", "Serie-A", "MLS", "LaLiga", "Bundesliga", "Ligue1"],
    # Limit of odds time limit in minutes towards start of the match as determined in Tableau
    "odds_limit" : [30240, 20160, 10080, 20160, 30240, 20160]
}

# List for companies
company_list = ["Bet365", "WilliamHill", "Bwin", "12Bet"]

In [3]:
# To concatenate with league name for csv file to be read
csv_concat = "_S2018-2019.csv"

In [4]:
# .......... FUNCTION TO BUILD CLASSIFIER FOR NEURAL NETWORK MODEL .......... #
def build_classifier(optimizer):
    classifier = Sequential()

    # Hidden layer
    classifier.add(Dense(units=3,     # Unit determination see below
                         activation="relu",
                         input_dim=len(feature_dict[f])                         
                        ))
    # Output layer
    classifier.add(Dense(units=3,     # Output
                         activation="softmax"
                        ))
    # Compile the neural work
    classifier.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return classifier
# .......... FUNCTION ENDS .......... #

'''
NUMBER OF HIDDEN NEURONS IN NEURAL NETWORK MODEL

According to the ref:
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
1. Layer of hidden nodes is set to 1
2. Number of input neurons is the length of features (4)
3. Number of output neurons is 3
4. Number of hidden neurons (hn) is supposed to be selected between 3 (output) and 4 (input), exclusive

In this study, the unit is set as 3
'''

In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier    # Random Forest (RF) model
from sklearn.svm import SVC    # Support Vector Machine (SVM) model
from sklearn.neighbors import KNeighborsClassifier    # K Nearest Neighbors (KNN) model
from tensorflow.keras.models import Sequential   # Neural Network model
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from numpy.random import seed
import pickle

# DataFrame to store data distribution for viz
distr_df = pd.DataFrame(columns=["league", "win_odds_s", "win_odds_k", "draw_odds_s", "draw_odds_k",
                                "lose_odds_s", "lose_odds_k", "odds_delta_time_s", "odds_delta_time_k"])

# Dict to store established model info. before comparison
model_estb = {}
# Initialize "model_estb" structure
for league in league_dict["league"]:
    model_estb[league] = {}
    for company in company_list:
        model_estb[league][company] = {}
        for tgt in ["ori", "alt"]:
            model_estb[league][company][tgt] = {}
            for f in ["f4", "f3"]:
                model_estb[league][company][tgt][f] = {}
                for model in ["rf", "svm", "knn", "nn"]:
                    model_estb[league][company][tgt][f][model] = {}
                    
                    model_estb[league][company][tgt][f][model]["precision"] = {}
                    model_estb[league][company][tgt][f][model]["recall"] = {}
                    model_estb[league][company][tgt][f][model]["f1-score"] = {}

# Loop through leagues
for i in range(len(league_dict["league"])):
    
    # Read csv data
    data = pd.read_csv(f'../data/{league_dict["league"][i]}{csv_concat}') 

    # Index of "distr_df" used to append new data
    distr_index = len(distr_df)
    # Append league info. to "distr_index"
    distr_df.loc[distr_index, "league"] = league_dict["league"][i]

    # Loop through companies
    for company in company_list:
                       
        # Dict to store data divided by company
        data_company = {}
        # Divide data by company
        data_company["ori"] = data.loc[data["company"] == company, :]
        # Generate altenative company-divided data with odds time under the limit determined in Tableau
        data_company["alt"] = data_company["ori"].loc[data_company["ori"]["odds_delta_time"] <=\
            league_dict["odds_limit"][i], :]
                       
        # .......... Data Distribution .......... #
        # Dict for features
        feature_dict = {}
        # Columns to be considered as features for machine learning models
        feature_dict["f4"] = ["win_odds", "draw_odds", "lose_odds", "odds_delta_time"]
        # "According to corr analysis in Tableau, "lose_odds" might be redundant to serve as feature             
        feature_dict["f3"] = ["win_odds", "draw_odds", "odds_delta_time"]
        # Determine data distribution for features to be used to generate machine learning (ML) models
        for feature in feature_dict["f4"]:
            skewness = stats.describe(data_company["ori"][feature])[4]
            kurtosis = stats.describe(data_company["ori"][feature])[5]
            # Save skewness and kurtosis to "distr_df"           
            distr_df.loc[distr_index, f"{feature}_s"] = skewness
            distr_df.loc[distr_index, f"{feature}_k"] = kurtosis
                       
        ########################################################################################
        # BASED ON SKEWNESS AND KURTOSIS, THERE IS SELDOM FEATUER THAT IS NORMALLY DISTRIBUTED.#
        # WE WILL USE RANDOM FOREST, SVM, KNN, AND NEURAL NETWORK MODELS THEREAFTER            #
        ########################################################################################
                       
        # .......... Prep. for ML .......... #
        # Column to be used as label(target) of ML model
        for tgt in ["ori", "alt"]:
            target = data_company[tgt]["result"]
            # Target encoder
            t_encoder = LabelEncoder().fit(target.values)
            # Transform categorical labels to numerical
            target_encd = t_encoder.transform(target.values)
            # One-hot encoding labels
            target_encd_categ = to_categorical(target_encd)
        
            # Columns to be used as fetures of ML model
            for f in ["f4", "f3"]:
                features = data_company[tgt][feature_dict[f]]
                       
                # Set up train and test data for all machine learning models to be built in this project
                X_train, X_test, y_train, y_test = train_test_split(features, target_encd_categ, random_state =8)
                       
                # MinMaxScaler is preferred over StandardScaler for scaling features
                X_scaler = MinMaxScaler().fit(X_train)

                X_train_scaled = X_scaler.transform(X_train)
                X_test_scaled = X_scaler.transform(X_test)                               

                seed(8)  
                       
                # Dict to hold all info. with regard to machine learning models (RF, SVM, KNN, NN)
                model_dict = {
                    "rf": {},
                    "svm": {},
                    "knn": {},
                    "nn": {}
                }
                    
                # Classifier for RF, SVM, and KNN models
                model_dict["rf"]["classifier"] = RandomForestClassifier(random_state=8)
                model_dict["svm"]["classifier"] = SVC(random_state=8)
                model_dict["knn"]["classifier"] = KNeighborsClassifier()                       

                # Wrap Keras model so it can be used by scikit-learn
                model_dict["nn"]["classifier"] = KerasClassifier(build_fn=build_classifier, verbose=0)                            
                       
                # **************************************************************** #
                # Use default setting of classifiers for the first round screening #
                # For KNN, set k as even numbers that are less than 20             #
                # **************************************************************** #
                       
                # Default parameter for each classifier
                model_dict["rf"]["param_grid"] = {
                    'n_estimators': [10]
                }

                model_dict["svm"]["param_grid"] = {
                    'kernel': ["linear"]
                }

                model_dict["knn"]["param_grid"] = {
                    'n_neighbors': list(range(1, 20, 2))
                }


                model_dict["nn"]["param_grid"] = {
                    'optimizer': ["adam"]
                }                                              
                       
                # Loop through models             
                for model in list(model_dict.keys()):

                    # Print current iterating model
                    print("*" * 100)
                    print(f'Current iterating: {league_dict["league"][i]}|{company}|target.{tgt}|{f}|{model.upper()}')

                    # Create GridSearch estimator (unnecessary in this step as we are using default settings)
                    # Use it to pass "k" to KNN model under model iteration
                    model_dict[model]["estimator"] = GridSearchCV(
                        model_dict[model]["classifier"], 
                        model_dict[model]["param_grid"]
                    )

                    # SVM model can not take One-Hot-encoding label
                    if model == "svm":
                        y_train_im = np.argmax(y_train, axis=-1) # Tempprary y_train and y_test used for model fitting
                        y_test_im = np.argmax(y_test, axis=-1) 
                    else:
                        y_train_im = y_train 
                        y_test_im = y_test    

                    # Fit model using grid search estimator
                    if model == "nn":
                        # Shuffling is recommended for neural network model
                        model_dict[model]["estimator"].fit(X_train_scaled, y_train_im, shuffle=True)
                    else:
                        model_dict[model]["estimator"].fit(X_train_scaled, y_train_im)

                    # Inverse labels in "y_test"
                    # https://stackoverflow.com/questions/54143458/convert-categorical-data-back-to-numbers-using-keras-utils-to-categorical
                    y_test_invr = t_encoder.inverse_transform(np.argmax(y_test, axis=-1))

                    # Predict "y_pred"
                    y_pred = model_dict[model]["estimator"].predict(X_test_scaled)

                    # Inverse labels in "y_pred"
                    # Note that different from Sequential predict, which always returns the same datatype of "y_train_im"
                    # "predict" for KerasClassifier("nn") returns CLASS prediction for given test data (like "predict_classes for Sequential")
                    if (model == "svm") or (model == "nn"):
                        y_pred_invr = t_encoder.inverse_transform(y_pred)
                    else:
                        y_pred_invr = t_encoder.inverse_transform(np.argmax(y_pred, axis=-1))

                    # Append inversed "y_test" and "y_pred"
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["result_test"] = y_test_invr
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["result_pred"] = y_pred_invr

                    # Scores
                    # Note that KerasClassifier also has "score" (mean accuracy)
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["training_score"] =\
                       model_dict[model]["estimator"].score(X_train_scaled, y_train_im)
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["testing_score"] =\
                       model_dict[model]["estimator"].score(X_test_scaled, y_test_im)
                       
                    # Classification report
                    classif = classification_report(y_test_invr, y_pred_invr).split(" ")
                    while "" in classif:
                       classif.remove("")                    
                                      
                    # Store "precision", "recall", and "f1-score" info for "DRAW", "LOSE", and "WIN"
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["precision"]["DRAW"] = float(classif[5])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["recall"]["DRAW"] = float(classif[6])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["f1-score"]["DRAW"] = float(classif[7])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["precision"]["LOSE"] = float(classif[10])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["recall"]["LOSE"] = float(classif[11])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["f1-score"]["LOSE"] = float(classif[12])                          
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["precision"]["WIN"] = float(classif[15])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["recall"]["WIN"] = float(classif[16])
                    model_estb[league_dict["league"][i]][company][tgt][f][model]["f1-score"]["WIN"] = float(classif[17])                                              
                       
                    # Store models 
                    if model == "nn":
                       model_estb[league_dict["league"][i]][company][tgt][f][model]["model"] =\
                           model_dict[model]["estimator"].best_estimator_.model
                    else:
                       model_estb[league_dict["league"][i]][company][tgt][f][model]["model"] =\
                           model_dict[model]["estimator"]
                       
                    # Print complete information
                    print(f'{league_dict["league"][i]}|{company}|target.{tgt}|{f}|{model.upper()} DONE!!!\n')                       
                                              
# Save feature distribution data as csv file
distr_df.to_csv(f"../data/data_for_viz/data_distribution.csv", index=False, header=True)

Using TensorFlow backend.
C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


****************************************************************************************************
Current iterating: PremierLeague|Bet365|target.ori|f4|RF
PremierLeague|Bet365|target.ori|f4|RF DONE!!!

****************************************************************************************************
Current iterating: PremierLeague|Bet365|target.ori|f4|SVM


C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


PremierLeague|Bet365|target.ori|f4|SVM DONE!!!

****************************************************************************************************
Current iterating: PremierLeague|Bet365|target.ori|f4|KNN


W0729 23:35:28.432992  7324 deprecation.py:506] From C:\Users\kouda\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Ligue1|12Bet|target.ori|f3|NN DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f4|RF
Ligue1|12Bet|target.alt|f4|RF DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f4|SVM
Ligue1|12Bet|target.alt|f4|SVM DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f4|KNN
Ligue1|12Bet|target.alt|f4|KNN DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f4|NN


C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Ligue1|12Bet|target.alt|f4|NN DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f3|RF
Ligue1|12Bet|target.alt|f3|RF DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f3|SVM
Ligue1|12Bet|target.alt|f3|SVM DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f3|KNN
Ligue1|12Bet|target.alt|f3|KNN DONE!!!

****************************************************************************************************
Current iterating: Ligue1|12Bet|target.alt|f3|NN
Ligue1|12Bet|target.alt|f3|NN DONE!!!



C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [61]:
# Dicts to transfer corresponding data
training_score = {}
testing_score = {}

precision_draw = {}
precision_lose = {}
precision_win = {}
recall_draw = {}
recall_lose = {}
recall_win = {}
f1_draw = {}
f1_lose = {}
f1_win = {}

# Save corresponding data to dicts
for league in league_dict["league"]:
    for company in company_list:
        for tgt in ["ori", "alt"]:
            for f in ["f4", "f3"]:
                for model in list(model_dict.keys()):
                    training_score[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["training_score"]
                    testing_score[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["testing_score"]

                    precision_draw[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["precision"]["DRAW"]
                    precision_lose[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["precision"]["LOSE"]
                    precision_win[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["precision"]["WIN"] 
                    recall_draw[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["recall"]["DRAW"]
                    recall_lose[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["recall"]["LOSE"]
                    recall_win[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["recall"]["WIN"]  
                    f1_draw[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["f1-score"]["DRAW"]
                    f1_lose[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["f1-score"]["LOSE"]
                    f1_win[f'{league}_{company}_{tgt}_{f}_{model}'] =\
                        model_estb[league][company][tgt][f][model]["f1-score"]["WIN"]                 

In [12]:
# Convert score data into pandas DataFrame
score_df = pd.DataFrame()
score_df["param"] = list(training_score.keys())
score_df["training_score"] = list(training_score.values())
score_df["testing_score"] = list(testing_score.values())
score_df["precision_draw"] = list(precision_draw.values())
score_df["precision_lose"] = list(precision_lose.values())
score_df["precision_win"] = list(precision_win.values())
score_df["recall_draw"] = list(recall_draw.values())
score_df["recall_lose"] = list(recall_lose.values())
score_df["recall_win"] = list(recall_win.values())
score_df["f1_draw"] = list(f1_draw.values())
score_df["f1_lose"] = list(f1_lose.values())
score_df["f1_win"] = list(f1_win.values())
# View the shape of "score_df"
score_df

,param,training_score,testing_score,precision_draw,precision_lose,precision_win,recall_draw,recall_lose,recall_win,f1_draw,f1_lose,f1_win
0,PremierLeague_Bet365_ori_f4_rf,0.949299,0.540825,0.35,0.64,0.71,0.48,0.61,0.61,0.41,0.62,0.66
1,PremierLeague_Bet365_ori_f4_svm,0.574973,0.552142,0.00,0.64,0.52,0.00,0.46,0.88,0.00,0.53,0.66
2,PremierLeague_Bet365_ori_f4_knn,0.999191,0.616006,0.45,0.61,0.70,0.42,0.68,0.66,0.44,0.64,0.68
3,PremierLeague_Bet365_ori_f4_nn,0.426915,0.425222,0.00,0.26,0.44,0.00,0.05,0.91,0.00,0.08,0.59
4,PremierLeague_Bet365_ori_f3_rf,0.948490,0.529507,0.34,0.64,0.70,0.47,0.62,0.60,0.39,0.63,0.64
5,PremierLeague_Bet365_ori_f3_svm,0.574434,0.552142,0.00,0.64,0.52,0.00,0.46,0.88,0.00,0.54,0.66
6,PremierLeague_Bet365_ori_f3_knn,0.998921,0.619240,0.46,0.61,0.70,0.42,0.68,0.67,0.44,0.64,0.68
7,PremierLeague_Bet365_ori_f3_nn,0.464401,0.451900,0.00,0.00,0.45,0.00,0.00,1.00,0.00,0.00,0.62
8,PremierLeague_Bet365_alt_f4_rf,0.951459,0.564186,0.37,0.68,0.73,0.55,0.56,0.67,0.44,0.61,0.70
9,PremierLeague_Bet365_alt_f4_svm,0.564767,0.581357,0.00,0.66,0.56,0.00,0.45,0.89,0.00,0.53,0.69


In [13]:
# Save score data as csv file
score_df.to_csv(f"../data/data_for_viz/ml_scores.csv", index=False, header=True)

##### After viz and analysed in Tableau, the following models were selected for further fine tuning:
    PremierLeague_12Bet_ori_f4_knn
    LaLiga_12Bet_ori_f4_knn
    Bundesliga_12Bet_alt_f4_knn
    Serie-A_12Bet_alt_f4_knn
    Ligue1_12Bet_alt_f4_knn
    MLS_12Bet_alt_f4_knn
    PremierLeague_WilliamHill_ori_f4_knn (Alternative model from another betting company other than 12Bet)
    
    From the results:
    1. 12Bet is the best betting company we might want to retrieve odds-movement data from
    2. All selected models were KNN with n_neighbors=5
    3. Removal of odds that were too "old" facilitates for two thirds of leagues in building up ML models
    4. Despite a high correlation with "draw_odds" (and "win_odds" to some point), retaining "lose_odds" as feature is benefitial in building up ML models

In [30]:
# List to hold selected ML model params
model_sel = [
    "PremierLeague_12Bet_ori_f4_knn",
    "LaLiga_12Bet_ori_f4_knn",
    "Bundesliga_12Bet_alt_f4_knn",
    "Serie-A_12Bet_alt_f4_knn",
    "Ligue1_12Bet_alt_f4_knn",
    "MLS_12Bet_alt_f4_knn",
    "PremierLeague_WilliamHill_ori_f4_knn"
]

In [56]:
# Set up tuning parameters for each classifier
param_grid = {
    'weights': ["uniform", "distance"],
    'algorithm': ["ball_tree", "kd_tree", "brute", "auto"],
    'leaf_size': [30, 300]
}

In [66]:
# Dict to hold tuning info.
tuning_dict = {}

# Loop through names in "model_sel"
for name in model_sel:
    
    # Retrieve params from iterated name
    [league, company, tgt, f, model] = name.split("_")
    # Retrieve corresponding model from "model_estb"
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    # Prep. "tuning_dict" to store tuning results
    tuning_dict[f"{league}_{company}"] = {}
    tuning_dict[f"{league}_{company}"]["precision"] = {}    
    tuning_dict[f"{league}_{company}"]["recall"] = {}
    tuning_dict[f"{league}_{company}"]["f1-score"] = {}    
    
    # Retrieve the position of league from "league_dict['league']"
    for idx in range(len(league_dict["league"])):
        if league_dict["league"][idx] == league:
            break
    
    # Read csv data
    data = pd.read_csv(f'../data/{league}{csv_concat}')    
    # Retrieve data from company indicated in model
    data_co = data.loc[data["company"] == company, :]
    
    # Determine data used for fine tuning our machine models
    if league == "PremierLeague" or league == "LaLiga":        
        data_tuning = data_co        
    else:
        data_tuning = data_co.loc[data_co["odds_delta_time"] <= league_dict["odds_limit"][idx], :]
    
    # Column to be used as label(target)
    target_tuning = data_tuning["result"]

    # Target encoder
    t_encoder_tuning = LabelEncoder().fit(target_tuning.values)
    # Transform categorical labels to numerical
    target_tuning_encd = t_encoder_tuning.transform(target_tuning.values)       
    # One-hot encoding labels
    target_tuning_encd_categ = to_categorical(target_tuning_encd)

    # Columns to be used as fetures of ML model
    features_tuning = data_tuning[feature_dict["f4"]]
    
    # Set up train and test data for all machine learning models to be built in this project
    X_train, X_test, y_train, y_test = train_test_split(features_tuning, target_tuning_encd_categ, random_state =8)

    # MinMaxScaler is preferred over StandardScaler for scaling features
    X_scaler = MinMaxScaler().fit(X_train)

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)    
    
    # Print current iterating model
    print("*" * 100)
    print(f'Current iterating: {league}_{company}_ML_model')
    
    # Create GridSearch estimator
    model_estimator = GridSearchCV(knn_classifier, param_grid)
    
    # Fit model using grid search estimator
    model_estimator.fit(X_train_scaled, y_train)
    
    # Inverse labels in "y_test"
    y_test_invr = t_encoder_tuning.inverse_transform(np.argmax(y_test, axis=-1))
    
    # Predict "y_pred"
    y_pred = model_estimator.predict(X_test_scaled)
    # Inverse labels in "y_pred"
    y_pred_invr = t_encoder_tuning.inverse_transform(np.argmax(y_pred, axis=-1))
    
    # Append inversed "y_test" and "y_pred"
    tuning_dict[f"{league}_{company}"]["result_test"] = y_test_invr
    tuning_dict[f"{league}_{company}"]["result_pred"] = y_pred_invr
    
    # Scores
    tuning_dict[f"{league}_{company}"]["training_score"] = model_estimator.score(X_train_scaled, y_train)
    tuning_dict[f"{league}_{company}"]["testing_score"] = model_estimator.score(X_test_scaled, y_test)
    
    # Classification report
    classif = classification_report(y_test_invr, y_pred_invr).split(" ")
    while "" in classif:
        classif.remove("")
        
    # Store "precision", "recall", and "f1-score" info for "DRAW", "LOSE", and "WIN"
    tuning_dict[f"{league}_{company}"]["precision"]["DRAW"] = float(classif[5])
    tuning_dict[f"{league}_{company}"]["recall"]["DRAW"] = float(classif[6])
    tuning_dict[f"{league}_{company}"]["f1-score"]["DRAW"] = float(classif[7])
    tuning_dict[f"{league}_{company}"]["precision"]["LOSE"] = float(classif[10])
    tuning_dict[f"{league}_{company}"]["recall"]["LOSE"] = float(classif[11])
    tuning_dict[f"{league}_{company}"]["f1-score"]["LOSE"] = float(classif[12])                          
    tuning_dict[f"{league}_{company}"]["precision"]["WIN"] = float(classif[15])
    tuning_dict[f"{league}_{company}"]["recall"]["WIN"] = float(classif[16])
    tuning_dict[f"{league}_{company}"]["f1-score"]["WIN"] = float(classif[17])
    
    # Save model as .pkl file (documentation)
    with open(f"../models/{league}_{company}_ML_model.pkl", 'wb') as fid:
        pickle.dump(model_estimator, fid)

    # Print complete information
    print(f'Current iterating: {league}_{company}_ML_model DONE!!!\n')     
        

C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


****************************************************************************************************
Current iterating: PremierLeague_12Bet_ML_model
Current iterating: PremierLeague_12Bet_ML_model DONE!!!



C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


****************************************************************************************************
Current iterating: LaLiga_12Bet_ML_model
Current iterating: LaLiga_12Bet_ML_model DONE!!!



C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


...

****************************************************************************************************
Current iterating: Serie-A_12Bet_ML_model
Current iterating: Serie-A_12Bet_ML_model DONE!!!



C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


****************************************************************************************************
Current iterating: Ligue1_12Bet_ML_model
Current iterating: Ligue1_12Bet_ML_model DONE!!!

****************************************************************************************************
Current iterating: MLS_12Bet_ML_model
Current iterating: MLS_12Bet_ML_model DONE!!!



C:\Users\kouda\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


****************************************************************************************************
Current iterating: PremierLeague_WilliamHill_ML_model
Current iterating: PremierLeague_WilliamHill_ML_model DONE!!!



In [26]:
# Save X-scaler and target_encoder
x_scalerfile = '../models/X-scaler.sav'
pickle.dump(X_scaler, open(x_scalerfile, 'wb'))

y_scalerfile = '../models/target-encoder.sav'
pickle.dump(t_encoder_tuning, open(y_scalerfile, 'wb'))

In [76]:
# Convert score data into pandas DataFrame
final_score_df = pd.DataFrame()

# Dict to show comparison results between test and pred
comparison = {}
# Loop through "tuning_dict"
for i in range(len(tuning_dict)):
    final_score_df.loc[i, "param"] = list(tuning_dict.keys())[i]
    final_score_df.loc[i, "training_score"] = list(tuning_dict.values())[i]["training_score"]
    final_score_df.loc[i, "testing_score"] = list(tuning_dict.values())[i]["testing_score"]
    
    final_score_df.loc[i, "precision_win"] = list(tuning_dict.values())[i]["precision"]["WIN"]    
    final_score_df.loc[i, "precision_draw"] = list(tuning_dict.values())[i]["precision"]["DRAW"]
    final_score_df.loc[i, "precision_lose"] = list(tuning_dict.values())[i]["precision"]["LOSE"]    
    final_score_df.loc[i, "recall_win"] = list(tuning_dict.values())[i]["recall"]["WIN"]    
    final_score_df.loc[i, "recall_draw"] = list(tuning_dict.values())[i]["recall"]["DRAW"]
    final_score_df.loc[i, "recall_lose"] = list(tuning_dict.values())[i]["recall"]["LOSE"]    
    final_score_df.loc[i, "f1_win"] = list(tuning_dict.values())[i]["f1-score"]["WIN"]    
    final_score_df.loc[i, "f1_draw"] = list(tuning_dict.values())[i]["f1-score"]["DRAW"]
    final_score_df.loc[i, "f1_lose"] = list(tuning_dict.values())[i]["f1-score"]["LOSE"]
    
    # Print the comparison between "test" and "pred" for every model
    print("*" * 50)
    print(f'{list(tuning_dict.keys())[i]}_test: {list(tuning_dict.values())[i]["result_test"]}')
    print(f'{list(tuning_dict.keys())[i]}_pred: {list(tuning_dict.values())[i]["result_pred"]}\n')          


# View "final_score_df"
final_score_df

**************************************************
PremierLeague_12Bet_test: ['L' 'W' 'W' ... 'D' 'L' 'D']
PremierLeague_12Bet_pred: ['L' 'W' 'W' ... 'D' 'L' 'D']

**************************************************
LaLiga_12Bet_test: ['L' 'D' 'L' ... 'W' 'W' 'W']
LaLiga_12Bet_pred: ['L' 'D' 'L' ... 'W' 'D' 'W']

**************************************************
Bundesliga_12Bet_test: ['W' 'W' 'D' ... 'L' 'D' 'L']
Bundesliga_12Bet_pred: ['W' 'W' 'W' ... 'L' 'D' 'D']

**************************************************
Serie-A_12Bet_test: ['D' 'L' 'W' ... 'W' 'L' 'L']
Serie-A_12Bet_pred: ['D' 'D' 'W' ... 'W' 'L' 'L']

**************************************************
Ligue1_12Bet_test: ['W' 'W' 'D' ... 'D' 'L' 'W']
Ligue1_12Bet_pred: ['W' 'W' 'D' ... 'D' 'L' 'D']

**************************************************
MLS_12Bet_test: ['D' 'W' 'D' ... 'W' 'W' 'W']
MLS_12Bet_pred: ['D' 'W' 'D' ... 'W' 'W' 'W']

**************************************************
PremierLeague_WilliamHill_test:

,param,training_score,testing_score,precision_win,precision_draw,precision_lose,recall_win,recall_draw,recall_lose,f1_win,f1_draw,f1_lose
0,PremierLeague_12Bet,0.998752,0.831877,0.87,0.71,0.86,0.88,0.73,0.83,0.88,0.72,0.85
1,LaLiga_12Bet,0.997878,0.811596,0.87,0.77,0.82,0.84,0.82,0.79,0.85,0.80,0.81
2,Bundesliga_12Bet,0.998576,0.811635,0.86,0.74,0.84,0.85,0.79,0.81,0.85,0.77,0.83
3,Serie-A_12Bet,0.998332,0.801112,0.85,0.75,0.81,0.84,0.79,0.78,0.85,0.77,0.79
4,Ligue1_12Bet,0.998386,0.760323,0.83,0.69,0.78,0.81,0.75,0.74,0.82,0.72,0.76
5,MLS_12Bet,0.997033,0.746306,0.81,0.64,0.74,0.83,0.65,0.69,0.82,0.65,0.71
6,PremierLeague_WilliamHill,0.998170,0.791641,0.83,0.67,0.84,0.83,0.70,0.81,0.83,0.68,0.82


In [77]:
# Save score data as csv file
final_score_df.to_csv(f"../data/data_for_viz/ml_scores_final.csv", index=False, header=True)